# Sensor Data Fusion

Lecturer: Prof. Baum \
Tutor: Shishan Yang and Laura Wolf\
Semester: Winter 20/21

## Homework 2

GPS consists of  24 satellites in  orbit (20200km above mean sea level). 

Each  satellite broadcasts its location (in spherical coordinates $[\theta, \phi, r]^T$) plus the emission time (see figures below).

A GPS device receives at time $t=0s$ the following four satellite signals:

| $i$ | $p_i$ | $t_i$ |
| :--- | :---: | ---: |
| 1 | $[  0^{°},  40^{°}, 20200 \text{km}]^T$ | -67.603 ms |
| 2 | $[ 10^{°},  20^{°}, 20200 \text{km}]^T$ | -70.102 ms |
| 3 | $[ 10^{°}, -10^{°}, 20200 \text{km}]^T$ | -78.690 ms |
| 4 | $[ -10^{°}, -20^{°}, 20200 \text{km}]^T$ | -82.942 ms |

Assume that the speed-of-light is $c=3\cdot10^8\text{ms}^{-1}$ and the Earth is an ideal sphere with radius $r^{\text{E}}=6370 \text{km}$.

![Satellite Visuals](https://owncloud.gwdg.de/index.php/s/guLnIkLe4aOcL1E/download)

---
The following tasks will have missing sections marked that you should fill out. 

Missing code parts are marked by
```
# ... code code code
=== YOUR CODE HERE ===

=== END OF YOUR CODE ===
# ... code code code
```
If you are asked to implement a function, make sure to check what variable will be returned by the function and to fill it accordingly. Do not change code outside of the indicated sections.

Furthermore, some questions require theoretical answers instead of python code.

Such questions will have a field marked like this: 

=== YOUR ANSWER HERE === 

In [84]:
# import statements
import numpy as np
import matplotlib.pyplot as plt
import math

In [85]:
# Definition of global variables
speed_light = 3 * math.pow(10, 5)  # Speed of light def. - 10^5 because we want km/s
R = 6370  # earth radius (assuming earth is a perfect sphere)
t = np.array([67.603e-3, 70.102e-3, 78.690e-3, 82.942e-3])  # emission times

# location in spherical coordinates
p = [
    [0, 40, 20200],
    [10, 20, 20200],
    [10, -10, 20200],
    [-10,-20, 20200]
    ]
p = np.array(p)
spher_coor_p = p  # add a second alias...

---
### a)
Write a function which converts sphere coordinates $(\theta,\phi,r)$ into Cartesian coordinates $(x,y,z)$

Hints: 
- The $x$ coordinates could be obtained according to $x = (r^\text{E}+r)\,cos\phi\,cos\theta$
- You could use `math.radians(...)` to convert degrees to radians

In [90]:
def sphere_to_cartesian(p_sphere, R):
    """
    Function that converts spherical to cartesian coordinates. 
    :param p_sphere: 3x1 vector [theta, phi, R] with
    theta, longitude in degree
    phi, latitude in degree
    r, altitude in km
    :param R: earth radius in km
    :return: p_cartesian, converted cartesian coordinates, as 3x1 vector [x,y,z]
    """
    # make sure everything is numpy
    p_sphere = np.array(p_sphere)
    
    # === YOUR CODE HERE ===
    theta = np.deg2rad(p_sphere[0]) # Strich horizontal
    phi = np.deg2rad(p_sphere[1])   # Strich vertikal
    radius = p_sphere[2]

    # approach using rotation matrices
    # point = np.array([0, 0, R + radius])
    # rotation_y = np.array([[np.cos(phi), 0, np.sin(phi)], [0, 1, 0], [-np.sin(phi), 0, np.cos(phi)]])
    # rotation_z = np.array([[1, 0, 0], [0, np.cos(theta), -np.sin(theta)], [0, np.sin(theta), np.cos(theta)]])
    # p_cartesian = rotation_z.dot(rotation_y.dot(point))

    x = (radius + R) * np.cos(phi) * np.cos(theta)
    y = (radius + R) * np.cos(phi) * np.sin(theta)
    z = (radius + R) * np.sin(phi)

    p_cartesian = [x, y, z]

    # === END OF YOUR CODE ===
    
    return p_cartesian

Using the function you implemented, the Cartesian coordinates of these four Satellites will be calculated:

In [91]:
# calculating the cartesian coordinates based on your function:
cart_coor_p = []
for location in spher_coor_p:
    cart_coor_p.append(sphere_to_cartesian(location, R))
    
# take a look at the results:
print("Resulting cartesian Coordinates of the Satellite Locations:")
for location in cart_coor_p:
    print(location)

Resulting cartesian Coordinates of the Satellite Locations:
[20353.800853671244, 0.0, 17078.866789371346]
[24588.318488043453, 4335.58395969483, 9087.475208163018]
[25768.81646714079, 4543.737604081509, -4613.832080610339]
[24588.318488043453, -4335.58395969483, -9087.475208163018]


---
### b)
Please calculate the distance between satellites and GPS device.

In [81]:
d = np.zeros((4,))

# Use the following section to fill d with the distances between satellites and the GPS device.
# === YOUR CODE HERE ===

d = np.array([])
for current_emission_time in t:
    d = np.append(d, current_emission_time * speed_light)

# === END OF YOUR CODE ===

assert d.shape == (4,)  # make sure that d still has the correct shape
print("Distances between satellites and GPS device:")
for distance in d:
    print(distance)

Distances between satellites and GPS device:
20280.899999999998
21030.6
23607.0
24882.600000000002


Using the distance measurements, form the measurement equation like we did in the lecture.

=== YOUR ANSWER HERE ===
see slide 27 - 'individual measurement equation'


### c)

Reformulate the non-linear measurement equation in b) into a linear measurement equation.

=== YOUR ANSWER HERE ===

see slide 27 - 'individual measurement equation'
Now, implement your reformulation so that the least squares estimate can be calculated.

In [98]:
cart_coor_p = np.array(cart_coor_p)

# You will now need to create y and H of the measurement equation.
#    y should be of shape (3,)
#    H should be of shape (3,3)
# === YOUR CODE HERE ===

y = [d[0] ** 2 - d[3] ** 2 - np.linalg.norm(cart_coor_p[0]) ** 2 + np.linalg.norm(cart_coor_p[3]) ** 2,
    d[1] ** 2 - d[3] ** 2 - np.linalg.norm(cart_coor_p[1]) ** 2 + np.linalg.norm(cart_coor_p[3]) ** 2,
    d[2] ** 2 - d[3] ** 2 - np.linalg.norm(cart_coor_p[2]) ** 2 + np.linalg.norm(cart_coor_p[3]) ** 2]

H = 2 * np.array([
    [cart_coor_p[3][0] - cart_coor_p[0][0], cart_coor_p[3][1] - cart_coor_p[0][1], cart_coor_p[3][2] - cart_coor_p[0][2]],
    [cart_coor_p[3][0] - cart_coor_p[1][0], cart_coor_p[3][1] - cart_coor_p[1][1], cart_coor_p[3][2] - cart_coor_p[1][2]],
    [cart_coor_p[3][0] - cart_coor_p[2][0], cart_coor_p[3][1] - cart_coor_p[2][1], cart_coor_p[3][2] - cart_coor_p[2][2]]
])

# === END OF YOUR CODE ===

y = np.array(y)
H = np.array(H)

# ensure everything is in the correct form
assert y.shape == (3,)
assert H.shape == (3,3)

Now, the least squares solution will be calculated using your y and H from above.

In [68]:
x_hat = np.linalg.inv(H.T @ H) @ H.T @ y 

print("Least squares solution:")
for v in x_hat:
    print(v)

Least squares solution:
4940.480071646369
-985.599855813158
5685.745152681407


### d)
Write a function which converts the cartesian coordinates into sphere coordinates. Use the function you implemented, calculate the longitude and latitude of the GPS device, and find out where it is using using _GPS Visualizer_ :

[http://www.gpsvisualizer.com/map?form=google](http://www.gpsvisualizer.com/map?form=google)

In [96]:
def cartesian_to_sphere(p_cartesian, R):
    """
    :param p_cartesian: Point represented in cartesian coordinates as 3x1 vector
    :param R: Earth Radius in km
    :return: p_sphere, point represented in sphere coordinates, 
            3x1 vector: [theta, phi,r],  with
            theta, longitude in degree
            phi, latitude in degree
            r, altitude in km
    """
    p_cartesian = np.array(p_cartesian)
    
    # === YOUR CODE HERE ===
    x = p_cartesian[0]
    y = p_cartesian[1]
    z = p_cartesian[2]

    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arctan2(y, x)
    phi = np.arccos(z / r)

    p_sphere = [theta, phi, r]
    
    # === END OF YOUR CODE ===
    
    return p_sphere

Now, your function will be applied to acquire the resulting coordinates:

In [97]:
p_GPS = cartesian_to_sphere(x_hat, R)

print("Spherical Coordinates of the Least Squares Result:")
print("Latitude:",p_GPS[1])
print("Longitude:", p_GPS[0])
print("r:",p_GPS[2])

Spherical Coordinates of the Least Squares Result:
Latitude: 0.7250520168807857
Longitude: -0.19690969907852807
r: 7596.5418682025265


To import this data into gpsvisualizer, copy the output of the following cell and paste it into the "Paste your data here" box on the bottom right to see where the location is on earth:

In [ ]:
# formatting for gpsvisualizer
print("type,latitude,longitude")
print("W,{:f},{:f}".format(p_GPS[1], p_GPS[0]))